# Practicum 4.3
## Implementeer elk gemaakt ETL-schema in Python

In [1]:
import pandas as pd
import pyodbc
import sqlite3
from datetime import datetime

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_26760\2168663434.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [42]:
DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS', 
    'database': 'DEDS'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor   

# Extract

**Database connection**

In [43]:
go_sales_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_sales.sqlite')
go_crm_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_crm.sqlite')
go_staff_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_staff.sqlite')

**Dataframes aanmaken**

In [44]:
Go_staff_queries = {
    'Course': 'SELECT * FROM Course',
    'Training': 'SELECT * FROM Training',
    'Sales_staff': 'SELECT * FROM Sales_staff',
    'Satisfaction': 'SELECT * FROM Satisfaction',
    'Satisfaction_type': 'SELECT * FROM Satisfaction_type',
}

Go_crm_queries = {
    'Retailer_contact': 'SELECT * FROM Retailer_contact',
    'Retailer_site' : 'SELECT * FROM Retailer_site',
    'Retailer' : 'SELECT * FROM Retailer',
    'Retailer_type' : 'SELECT * FROM Retailer_type',
    'COUNTRY': 'SELECT * FROM COUNTRY',
    'Sales_territory': 'SELECT * FROM Sales_territory'
}

Go_sales_queries = {
    'Order_method': 'SELECT * FROM Order_method',
    'Order_header': 'SELECT * FROM Order_header',
    'Order_details': 'SELECT * FROM Order_details',
    'Returned_item': 'SELECT * FROM Returned_item',
    'Return_reason': 'SELECT * FROM Return_reason',
    'Product': 'SELECT * FROM Product',
    'PRODUCT_TYPE': 'SELECT * FROM PRODUCT_TYPE',
    'PRODUCT_LINE' : 'SELECT * FROM PRODUCT_LINE',
    'Sales_TARGETData' : 'SELECT * FROM Sales_TARGETData',
    'Sales_branch': 'SELECT * FROM Sales_branch'
}


dataframes = {}

# Lees elke tabel in een DataFrame
for table_name, query in Go_staff_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_staff_con)

for table_name, query in Go_crm_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_crm_con)

for table_name, query in Go_sales_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_sales_con)


#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df

Sales_staff['DATE_HIRED'] = pd.to_datetime(Sales_staff['DATE_HIRED'], errors='coerce')
Sales_staff['DATE_HIRED'] = Sales_staff['DATE_HIRED'].dt.date

print(Sales_TARGETData)

          Id SALES_STAFF_CODE SALES_YEAR SALES_PERIOD  \
0          1                6       2021            5   
1          2                6       2021            5   
2          3                6       2021            5   
3          4                6       2021            5   
4          5                6       2021            5   
...      ...              ...        ...          ...   
39525  39526               83       2020            8   
39526  39527               83       2021            3   
39527  39528               83       2021            3   
39528  39529               83       2021            3   
39529  39530               83       2021            3   

                 RETAILER_NAME PRODUCT_NUMBER SALES_TARGET RETAILER_CODE  \
0           Altitudes extrêmes             53          489            33   
1           Altitudes extrêmes             55          666            33   
2           Altitudes extrêmes             56          682            33   
3          

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_26760\2472730092.py:49: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Sales_staff['DATE_HIRED'] = pd.to_datetime(Sales_staff['DATE_HIRED'], errors='coerce')


# Transform & Load

### **Retailer_dimensie**

In [124]:
#Drop TRIAL columns want anders veel merge problemen.
dataframes_dict = {
    "Retailer_contact": Retailer_contact,
    "Retailer_site": Retailer_site,
    "COUNTRY": COUNTRY,
    "Sales_territory": Sales_territory,
    "Retailer": Retailer,
    "Retailer_type": Retailer_type
}

for name, df in dataframes_dict.items():
    # Drop any column that contains 'TRIAL'
    trial_cols = [col for col in df.columns if 'TRIAL' in col]
    df.drop(columns=trial_cols, inplace=True)

merge1 = pd.merge(Retailer_contact, Retailer_site, on = 'RETAILER_SITE_CODE')
merge2 = pd.merge(merge1, COUNTRY, on = 'COUNTRY_CODE')
merge3 = pd.merge(merge2, Sales_territory, on= 'SALES_TERRITORY_CODE')
merge4 = pd.merge(merge3, Retailer, on= 'RETAILER_CODE')
Retailer_dimensie = pd.merge(merge4, Retailer_type, on = 'RETAILER_TYPE_CODE')

def get_category(job_position):
    if 'Purchaser' in job_position:
        return 'Purchaser'
    elif 'Manager' in job_position:
        return 'Manager'
    else:
        return 'Other'

# Loop over elke rij en wijs categorie toe
categories = []
for index, row in Retailer_dimensie.iterrows():
    category = get_category(row['JOB_POSITION_EN'])
    categories.append(category)

# Voeg de categorieën toe als nieuwe kolom
Retailer_dimensie['Retailer_Position_category_Category'] = categories

Retailer_dimensie = Retailer_dimensie.rename(columns= {
    'RETAILER_CONTACT_CODE': 'Retailer_Retailer_contact_code',
    'FIRST_NAME': 'Retailer_FIRST_NAME',
    'LAST_NAME': 'Retailer_LAST_NAME',
    'E_MAIL': 'Retailer_E-mail',
    'ADDRESS1': 'Retailer_Address_ADDRESS1',
    'ADDRESS2': 'Retailer_Address_ADDRESS2',
    'POSTAL_ZONE': 'Retailer_Zone_POSTAL_ZONE',
    'CITY': 'Retailer_City_CITY',
    'REGION': 'Retailer_Region_REGION',
    'COUNTRY_CODE': 'Retailer_Country_COUNTRY_CODE',
    'COUNTRY_EN':'Retailer_Country_COUNTRY_EN',
    'SALES_TERRITORY_CODE': 'Retailer_Territory_TERRITORY_CODE',
    'TERRITORY_NAME_EN': 'Retailer_Territory_TERRITORY_NAME_EN',
    'GENDER': 'Retailer_Gender_GENDER',
    'RETAILER_CODE' : 'Retailer_Company_RETAILER_CODE',
    'COMPANY_NAME' : 'Retailer_Company_COMPANY_NAME',
    'RETAILER_TYPE_CODE': 'Retailer_Type_RETAILER_TYPE_CODE',
    'RETAILER_TYPE_EN': 'Retailer_Type_RETAILER_TYPE_EN',
    'JOB_POSITION_EN': 'Retailer_Position_JOB_POSITION_EN'

})
#print(Retailer_dimensie.columns)


In [125]:
for index, row in Retailer_dimensie.iterrows():
    try:
        query = """INSERT INTO Retailer VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
        params = (
            row['Retailer_Retailer_contact_code'], row['Retailer_FIRST_NAME'], row['Retailer_LAST_NAME'],
            row['Retailer_E-mail'], row['Retailer_Address_ADDRESS1'],
            str(row['Retailer_Address_ADDRESS2']).replace('\'', '\'\'') if pd.notna(row['Retailer_Address_ADDRESS2']) else None, 
            row['Retailer_Zone_POSTAL_ZONE'], row['Retailer_City_CITY'], row['Retailer_Region_REGION'],
            row['Retailer_Country_COUNTRY_CODE'], row['Retailer_Country_COUNTRY_EN'],
            row['Retailer_Territory_TERRITORY_CODE'], row['Retailer_Territory_TERRITORY_NAME_EN'],
            row['Retailer_Gender_GENDER'], row['Retailer_Company_RETAILER_CODE'],
            row['Retailer_Company_COMPANY_NAME'], row['Retailer_Type_RETAILER_TYPE_CODE'],
            row['Retailer_Type_RETAILER_TYPE_EN'], row['Retailer_Position_JOB_POSITION_EN'],
            row['Retailer_Position_category_Category']
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()

### **Sales_staff_dimensie**

In [62]:

merge1 = pd.merge(Sales_staff, Sales_branch, on= 'SALES_BRANCH_CODE', how= 'outer')
#print(merge1.columns)

merge2 = pd.merge(merge1, COUNTRY, on= 'COUNTRY_CODE', how= 'outer')
#print(merge2.columns)
Sales_staff_dimensie = pd.merge(merge2, Sales_territory, on= 'SALES_TERRITORY_CODE', how = 'outer')

#conversie om .year te gebruiken.
Sales_staff_dimensie['DATE_HIRED'] = pd.to_datetime(Sales_staff_dimensie['DATE_HIRED'])

# Gebruik .dt.year om het jaar van elke datum in de Series te extraheren
Sales_staff_dimensie['Sales_staff_In_dienst_nr'] = datetime.now().year - Sales_staff_dimensie['DATE_HIRED'].dt.year

for index, row in Sales_staff_dimensie.iterrows():
    aantal_jaar_in_dienst = row['Sales_staff_In_dienst_nr']

    if aantal_jaar_in_dienst < 20:
        Sales_staff_dimensie.at[index, 'Sales_staff_In_dienst_category_code'] = '<20 jaar'
    else:
        Sales_staff_dimensie.at[index, 'Sales_staff_In_dienst_category_code'] = '≥20 jaar'

Sales_staff_dimensie = Sales_staff_dimensie.rename(columns =  {
    'SALES_STAFF_CODE': 'Sales_staff_SALES_STAFF_CODE',
    'FIRST_NAME': 'Sales_staff_FIRST_NAME',
    'LAST_NAME': 'Sales_staff_LAST_NAME',
    'POSITION_EN': 'Sales_staff_Position_POSITION_EN',
    'EMAIL': 'Sales_staff_EMAIL',
    'MANAGER_CODE': 'Sales_staff_Manager_MANAGER_CODE',
    'SALES_BRANCH_CODE': 'Sales_staff_Branch_SALES_BRANCH_CODE',
    'ADDRESS1': 'Sales_staff_ADDRESS_ADDRESS1',
    'ADDRESS2': 'Sales_staff_ADDRESS_ADDRESS2',
    'CITY': 'Sales_staff_City_CITY',
    'REGION' : 'Sales_staff_Region_REGION',
    'POSTAL_ZONE': 'Sales_staff_Zone_POSTAL_ZONE',
    'COUNTRY_CODE': 'Sales_staff_Country_COUNTRY_CODE',
    'COUNTRY_EN': 'Sales_staff_Country_COUNTRY_EN',
    'SALES_TERRITORY_CODE': 'Sales_staff_Territory_TERRITORY_CODE',
    'TERRITORY_NAME_EN': 'Sales_staff_Territory_TERRITORY_NAME_EN'
})
#print(Sales_staff_dimensie)

In [127]:

for index, row in Sales_staff_dimensie.iterrows():
    try:
        query = """INSERT INTO Sales_staff VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
        params = (
            row['Sales_staff_SALES_STAFF_CODE'], row['Sales_staff_FIRST_NAME'], row['Sales_staff_LAST_NAME'],
            row['Sales_staff_EMAIL'], row['Sales_staff_Zone_POSTAL_ZONE'], row['Sales_staff_ADDRESS_ADDRESS1'],
            str(row['Sales_staff_ADDRESS_ADDRESS2']).replace('\'', '\'\'') if pd.notna(row['Sales_staff_ADDRESS_ADDRESS2']) else None, row['Sales_staff_City_CITY'], row['Sales_staff_Region_REGION'],
            row['Sales_staff_Country_COUNTRY_CODE'], row['Sales_staff_Country_COUNTRY_EN'],
            row['Sales_staff_Territory_TERRITORY_CODE'], row['Sales_staff_Territory_TERRITORY_NAME_EN'],
            row['Sales_staff_In_dienst_nr'], row['Sales_staff_In_dienst_category_code'],
            row['Sales_staff_Manager_MANAGER_CODE'], row['Sales_staff_Position_POSITION_EN'],
            row['Sales_staff_Branch_SALES_BRANCH_CODE']
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()


An error occurred: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 4 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')
INSERT INTO Sales_staff VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)


### **Product_dimensie**

In [17]:
merge1 = pd.merge(Product, PRODUCT_TYPE, on= 'PRODUCT_TYPE_CODE')
#print(merge1.columns)

Product_dimensie = pd.merge(merge1, PRODUCT_LINE, on = 'PRODUCT_LINE_CODE')
#print(merge2.columns)


for index, row in Product_dimensie.iterrows():
    Production_cost = float(row['PRODUCTION_COST'])

    if Production_cost < 100:
        Product_dimensie.at[index, 'Product_Category_cost_Category'] = '<100'
    else:
        Product_dimensie.at[index, 'Product_Category_cost_Category'] = '>100'


Product_dimensie = Product_dimensie.rename(columns = {
    'PRODUCT_NUMBER': 'Product_PRODUCT_NUMBER',
    'INTRODUCTION_DATE': 'Product_Introduction_date_INTRODUCTION_DATE',
    'PRODUCTION_COST': 'Product_Cost_PRODUCTION_COST',
    'MARGIN' : 'Product_Margin_MARGIN',
    'PRODUCT_NAME' : 'Product_PRODUCT_NAME',
    'DESCRIPTION' : 'Product_DESCRIPTION',
    'PRODUCT_TYPE_CODE' : 'Product_Type_PRODUCT_TYPE_CODE',
    'PRODUCT_TYPE_EN' : 'Product_Type_PRODUCT_TYPE_EN',
    'PRODUCT_LINE_CODE' : 'Product_Line_PRODUCT_LINE_CODE',
    'PRODUCT_LINE_EN' : 'Product_Line_PRODUCT_LINE_EN'
})

print(Product_dimensie)



    Product_PRODUCT_NUMBER Product_Introduction_date_INTRODUCTION_DATE  \
0                        1                                   15-2-2011   
1                       10                                   15-2-2011   
2                      100                                   15-2-2011   
3                      101                                  15-12-2019   
4                      102                                  10-12-2019   
..                     ...                                         ...   
110                     95                                   15-2-2011   
111                     96                                   15-2-2011   
112                     97                                    5-3-2013   
113                     98                                   15-2-2011   
114                     99                                   15-2-2011   

    Product_Type_PRODUCT_TYPE_CODE Product_Cost_PRODUCTION_COST  \
0                                1          

In [18]:
for index, row in Product_dimensie.iterrows():
    try:
        query = """INSERT INTO Product VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
        params = (
            row['Product_PRODUCT_NUMBER'], row['Product_PRODUCT_NAME'], row['Product_DESCRIPTION'], row['Product_Type_PRODUCT_TYPE_CODE'],
            row['Product_Type_PRODUCT_TYPE_EN'], row['Product_Line_PRODUCT_LINE_CODE'], row['Product_Line_PRODUCT_LINE_EN'],
            row['Product_Introduction_date_INTRODUCTION_DATE'], row['Product_Cost_PRODUCTION_COST'], row['Product_Category_cost_Category'],
            row['Product_Margin_MARGIN']
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()


### **Day_dimensie**

### **Course_dimensie**

In [36]:
query = 'SELECT * FROM course'

course_dimensie = pd.read_sql_query(query, go_staff_con)

# Sluit de database verbinding
go_staff_con.close()

# Toon de eerste paar rijen van de DataFrame om te bevestigen
print(course_dimensie.head())

# Hernoem de kolommen in de DataFrame
course_dimensie = course_dimensie.rename(columns={
    'COURSE_CODE': 'Course_COURSE_CODE',
    'COURSE_DESCRIPTION': 'Course_COURSE_DESCRIPTION'
})


for index, row in course_dimensie.iterrows():

    try:
        query= f"INSERT INTO Course VALUES ({row['Course_COURSE_CODE']}, '{row['Course_COURSE_DESCRIPTION']}' )"
        export_cursor.execute(query)
    except pyodbc.Error:
        print(query)


export_conn.commit()
export_cursor.close()


  COURSE_CODE COURSE_DESCRIPTION TRIAL633
0           1     GO Orientation        T
1           2   GO Communication        T
2           3         GO Sales 1        T
3           4         GO Sales 2        T
4           5     GO Marketing 1        T


### **Satisfaction_type_dimensie**

In [129]:
satisfaction_type_dimensie = pd.DataFrame({
    'Satisfaction_type_SATISFACTION_TYPE_CODE': [1, 2, 3, 4, 5],
    'Satisfaction_type_SATISFACTION_TYPE_DESCRIPTION': [
        'Not satisfied',
        'Less than satisfied',
        'Satisfied',
        'Very Satisfied',
        'More than satisfied'
    ]

})

### **Order_method_dimensie**

In [130]:
order_method_dimensie = pd.DataFrame({
    'Order_method_ORDER_METHOD_CODE': [1, 2, 3, 4, 5, 7, 8],
    'Order_method_ORDER_METHOD_EN': [
        'Fax',
        'Telephone',
        'Mail',
        'E-mail',
        'Web',
        'Sales visit',
        'Special'
    ]
})

### **Return_reason_dimensie**

In [25]:
Return_reason_dimensie = Return_reason

Return_reason_dimensie = Return_reason_dimensie.rename(columns= {
    'RETURN_REASON_CODE': 'Return_reason_RETURN_REASON_CODE',
    'RETURN_DESCRIPTION_EN': 'Return_reason_RETURN_DESCRIPTION_EN'
})

for index, row in Return_reason_dimensie.iterrows():
    try:
        query = """INSERT INTO Return_reason VALUES (?, ?)"""
        params = (
            row['Return_reason_RETURN_REASON_CODE'], row['Return_reason_RETURN_DESCRIPTION_EN']
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()


### **Returned_item_feit**

In [191]:
Returned_item_feit = pd.merge(Returned_item, Return_reason, on='RETURN_REASON_CODE')

Returned_item_feit['RETURN_QUANTITY'] = Returned_item_feit['RETURN_QUANTITY'].astype(float)


Returned_item_feit = Returned_item_feit.rename(columns = {
    'RETURN_CODE': 'Returned_item_RETURN_CODE',
    'RETURN_DATE': 'Day_time',
    'RETURN_QUANTITY' : 'Returned_item_RETURN_QUANTITY',
    'RETURN_REASON_CODE' : 'Return_reason_RETURN_REASON_CODE'
})

Return_average = Returned_item_feit.groupby('Return_reason_RETURN_REASON_CODE')['Returned_item_RETURN_QUANTITY'].mean().reset_index(name='Returned_item_RETURN_AVERAGE')

# Merge the average back into the main DataFrame
Returned_item_feit = Returned_item_feit.merge(Return_average, on='Return_reason_RETURN_REASON_CODE')

for index, row in Returned_item_feit.iterrows():
    try:
        query = """INSERT INTO Returned_item VALUES (?, ?, ?, ?, ?)"""
        params = (
            row['Returned_item_RETURN_CODE'], row['Day_time'], row['Returned_item_RETURN_QUANTITY'],
            row['Returned_item_RETURN_AVERAGE'], row['Return_reason_RETURN_REASON_CODE']
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()


### **Order_feit**

In [30]:
merge1 = pd.merge(Order_header, Order_details, on= 'ORDER_NUMBER')

merge2 = pd.merge(merge1, Retailer_contact, on = 'RETAILER_CONTACT_CODE')

merge3 = pd.merge(merge2, Product, on= 'PRODUCT_NUMBER')

Order_feit = pd.merge(merge3, Order_method, on = 'ORDER_METHOD_CODE')


# Calculate 'Order_OMZET'
Order_feit['Order_OMZET'] = Order_feit['QUANTITY'] * (pd.to_numeric(Order_feit['UNIT_PRICE']))

# Locally convert 'UNIT_PRICE' and 'UNIT_SALE_PRICE' to float for the calculation
Order_feit['Order_KORTING'] = (
    (pd.to_numeric(Order_feit['UNIT_PRICE'], errors='coerce') - 
     pd.to_numeric(Order_feit['UNIT_SALE_PRICE'], errors='coerce')) / 
    pd.to_numeric(Order_feit['UNIT_PRICE'], errors='coerce')
) * 100

Order_feit['Order_KORTING'] = Order_feit['Order_KORTING'].map(lambda x: f"{round(x)}%")

# Check the columns and data
print(Order_feit[['Order_OMZET', 'Order_KORTING']])

Order_feit = Order_feit.rename(columns={
    'ORDER_NUMBER': 'Order_ORDER_NUMBER',
    'ORDER_DATE': 'Day_date',
    'ORDER_DETAIL_CODE': 'Order_ORDER_DETAIL_CODE',
    'QUANTITY': 'Order_QUANTITY',
    'UNIT_COST': 'Order_UNIT_COST',
    'UNIT_PRICE': 'Order_UNIT_PRICE',
    'UNIT_SALE_PRICE': 'Order_UNIT_SALE_PRICE',
    'RETAILER_CONTACT_CODE': 'Retailer_Retailer_contact_code',
    'PRODUCT_NUMBER': 'Product_PRODUCT_NUMBER',
    'ORDER_METHOD_CODE': 'Order_method_ORDER_METHOD_CODE'
})



       Order_OMZET Order_KORTING
0          1253.00            6%
1          6951.98            8%
2          3025.56            6%
3          4537.00           10%
4          5012.02            9%
...            ...           ...
43058      2432.00           12%
43059       432.00           12%
43060      3200.00           12%
43061      3168.00            5%
43062      1008.00           18%

[43063 rows x 2 columns]


In [34]:
for index, row in Order_feit.iterrows():
    try:
        query = """INSERT INTO [Order] VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
        params = (
            row['Order_ORDER_NUMBER'],
            row['Order_ORDER_DETAIL_CODE'],
            row['Order_QUANTITY'],
            row['Order_UNIT_COST'],
            row['Order_UNIT_PRICE'],
            row['Order_UNIT_SALE_PRICE'],
            row['Order_OMZET'],  
            row['Order_KORTING'],
            row['Retailer_Retailer_contact_code'],
            row['Product_PRODUCT_NUMBER'],
            row['Order_method_ORDER_METHOD_CODE'],
            row['Day_date']
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()

### **Target_feit**

In [59]:
merge1 = pd.merge(Sales_TARGETData, Sales_staff, on = 'SALES_STAFF_CODE')

merge2 = pd.merge(merge1, Product, on = 'PRODUCT_NUMBER')

Target_feit = pd.merge(merge2, Retailer, on = 'RETAILER_CODE')

def dagen_in_maand(maand, jaar):
    # Aantal dagen in elke maand (standaard, geen schrikkeljaar)
    dagen = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if maand == 2:  # Februari
        # Controleer op schrikkeljaar
        is_schrikkeljaar = jaar % 4 == 0 and (jaar % 100 != 0 or jaar % 400 == 0)
        if is_schrikkeljaar:
            return 29
    return dagen[maand - 1]

Target_feit['Aantal_dagen'] = Target_feit.apply(lambda row: dagen_in_maand(int(row['SALES_PERIOD']), int(row['SALES_YEAR'])), axis = 1)

Target_feit['SALES_TARGET'] = pd.to_numeric(Target_feit['SALES_TARGET'], errors='coerce')

Target_feit['Target_DAILY'] = Target_feit['SALES_TARGET'] / Target_feit['Aantal_dagen']

Target_feit = Target_feit.rename(columns = {
    'Id': 'Target_Id',
    'SALES_YEAR' : 'Year_nr',
    'SALES_TARGET': 'Target_SALES_TARGET',
    'SALES_PERIOD': 'Target_SALES_PERIOD',
    'SALES_STAFF_CODE': 'Sales_staff_SALES_STAFF_CODE',
    'PRODUCT_NUMBER': 'Product_PRODUCT_NUMBER',
    'RETAILER_CODE': 'Retailer_company_retailer_code'
})


In [61]:
for index, row in Target_feit.iterrows():
    try:
        query = """INSERT INTO Target VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
        params = (
            row['Target_Id'],
            row['Year_nr'],
            row['Target_SALES_TARGET'],
            row['Target_SALES_PERIOD'],
            row['Target_DAILY'],
            row['Sales_staff_SALES_STAFF_CODE'],
            row['Product_PRODUCT_NUMBER'],  
            row['Retailer_company_retailer_code'],
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()

### **Forecast_feit**

In [ ]:
merge = 

### **Satisfaction_feit**

In [74]:
merge1 = pd.merge(Satisfaction, Sales_staff, on = 'SALES_STAFF_CODE')

Satisfaction_feit = pd.merge(merge1, Satisfaction_type, on = 'SATISFACTION_TYPE_CODE')

Satisfaction_feit = Satisfaction_feit.rename(columns = {
    'YEAR' : 'Year_nr',
    'SALES_STAFF_CODE' : 'Satisfaction_SALES_STAFF_CODE',
    'SATISFACTION_TYPE_CODE' : 'Satisfaction_type_SATISFACTION_TYPE_CODE'
})

print(Satisfaction_feit.columns)



Index(['Year_nr', 'Satisfaction_SALES_STAFF_CODE',
       'Satisfaction_type_SATISFACTION_TYPE_CODE', 'TRIAL633_x', 'FIRST_NAME',
       'LAST_NAME', 'POSITION_EN', 'WORK_PHONE', 'EXTENSION', 'FAX', 'EMAIL',
       'DATE_HIRED', 'SALES_BRANCH_CODE', 'MANAGER_CODE', 'TRIAL633_y',
       'SATISFACTION_TYPE_DESCRIPTION', 'TRIAL633'],
      dtype='object')


In [75]:
for index, row in Satisfaction_feit.iterrows():
    try:
        query = """INSERT INTO Satisfaction VALUES (?, ?, ?)"""
        params = (
            row['Year_nr'],
            row['Satisfaction_SALES_STAFF_CODE'],
            row['Satisfaction_type_SATISFACTION_TYPE_CODE'],
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()

### **Training_feit**

In [71]:
#andere soort join want 2 Sales_code
#of mischien de fk sales staff cde weghalen want anders komt gwn dubbel voor ?
merge1 = pd.merge(Training, Sales_staff, on = 'SALES_STAFF_CODE')

Training_feit = pd.merge(merge1, Course, on = 'COURSE_CODE')


Training_feit = Training_feit.rename(columns = {
    'YEAR' : 'Year_nr',
    'SALES_STAFF_CODE' : 'Training_Sales_staff_code',
    'COURSE_CODE' : 'Course_COURSE_CODE'
})

print(Training_feit.columns)

Index(['Year_nr', 'Training_Sales_staff_code', 'Course_COURSE_CODE',
       'TRIAL633_x', 'FIRST_NAME', 'LAST_NAME', 'POSITION_EN', 'WORK_PHONE',
       'EXTENSION', 'FAX', 'EMAIL', 'DATE_HIRED', 'SALES_BRANCH_CODE',
       'MANAGER_CODE', 'TRIAL633_y', 'COURSE_DESCRIPTION', 'TRIAL633'],
      dtype='object')


In [73]:
for index, row in Training_feit.iterrows():
    try:
        query = """INSERT INTO Training VALUES (?, ?, ?)"""
        params = (
            row['Year_nr'],
            row['Training_Sales_staff_code'],
            row['Course_COURSE_CODE']
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)

export_conn.commit()

### **Close**

In [192]:
export_cursor.close()

### Slowly changing dimensions

**Return_reason_dimensie :** 
SDC 1 want we willen nieuwe data updaten, maar we hoeven de oude data niet het is niet relevant.

In [ ]:
Return_reason_dimensie = Return_reason

Return_reason_dimensie = Return_reason_dimensie.rename(columns= {
    'RETURN_REASON_CODE': 'Return_reason_RETURN_REASON_CODE',
    'RETURN_DESCRIPTION_EN': 'Return_reason_RETURN_DESCRIPTION_EN'
})

# Voorbeeld data aanpassing
aanpassingen = [
    {"Return_reason_RETURN_REASON_CODE": 1, "Return_reason_RETURN_DESCRIPTION_EN": "Faulty product"},  
    {"Return_reason_RETURN_REASON_CODE": 6, "Return_reason_RETURN_DESCRIPTION_EN": "Product did not meet expectations"} 
]

for aanpassing in aanpassingen:
    code_exists = aanpassing['Return_reason_RETURN_REASON_CODE'] in Return_reason_dimensie['Return_reason_RETURN_REASON_CODE'].values
    
    if code_exists:
        # DataFrame bijwerken
        Return_reason_dimensie.loc[Return_reason_dimensie['Return_reason_RETURN_REASON_CODE'] == aanpassing['Return_reason_RETURN_REASON_CODE'], 'Return_reason_RETURN_DESCRIPTION_EN'] = aanpassing['Return_reason_RETURN_DESCRIPTION_EN']
        
        # SQL update
        query_update = """UPDATE Return_reason SET Return_reason_RETURN_DESCRIPTION_EN = ? WHERE Return_reason_RETURN_REASON_CODE = ?"""
        params_update = (aanpassing['Return_reason_RETURN_DESCRIPTION_EN'], aanpassing['Return_reason_RETURN_REASON_CODE'])
        export_cursor.execute(query_update, params_update)
    else:
        # Nieuwe rij aan DataFrame toevoegen
        new_row_df = pd.DataFrame([aanpassing])
        Return_reason_dimensie = pd.concat([Return_reason_dimensie, new_row_df], ignore_index=True)
        
        # SQL insert
        query_insert = """INSERT INTO Return_reason (Return_reason_RETURN_REASON_CODE, Return_reason_RETURN_DESCRIPTION_EN) VALUES (?, ?)"""
        params_insert = (aanpassing['Return_reason_RETURN_REASON_CODE'], aanpassing['Return_reason_RETURN_DESCRIPTION_EN'])
        export_cursor.execute(query_insert, params_insert)

export_conn.commit()

# Toon de aangepaste DataFrame
print(Return_reason_dimensie)